# Part 0: Extracting the table again

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(wikipedia_link).text

soup = BeautifulSoup(raw_wikipedia_page,'lxml')
#print(soup.prettify())


In [2]:
table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

# print(table)

for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
    if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'): 
        continue
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            continue
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        continue
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)
    

In [3]:
unique_p = set(Postcode)
# print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)


# Part 1: Pre-processing - Installing geocoder

In [4]:
import geocoder
print('geocoder has been successfully imported.')

geocoder has been successfully imported.


# Part 2: Processing - Extracting Latitude and Longtitude for each Postal code in Toronto

In [15]:
import json
latitude = []
longitude = []

for elem in Postcode_u:
    try:
        j='toronto,'+elem
        #url ='https://maps.googleapis.com/maps/api/geocode/json'
        url ="https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}".format(API_key,j)
        response = requests.get(url).json() # get response
        #print(response)
        geographical_data = response['results'][0]['geometry']['location'] # get geographical coordinates
        
        latitude.append(geographical_data['lat'])
        longitude.append(geographical_data['lng'])
    except:
        pass
        #g = geocoder.google('{}, Toronto, Ontario'.format(elem))
        #lat_lng_coords = g.latlng
        # print(lat_lng_coords)
    #print(latitude)
    #latitude.append(lat_lng_coords[0])
    #longitude.append(lat_lng_coords[1])
    #print(elem, 'is RECEIVED.')
    # print(lat_lng_coords[0])
    # print(lat_lng_coords[1])


# Part 3: Post-processing - Creating Pandas Dataframe

In [16]:
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u,
              'Latitude': latitude, 'Longitude':longitude}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto.to_csv('toronto_base.csv')
df_toronto.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M8X,Etobicoke,The Kingsway,43.653654,-79.506944
1,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493
2,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325
3,M6K,West Toronto,"Exhibition Place, Parkdale Village",43.636847,-79.428191
4,M4J,East York,East Toronto,43.685347,-79.338106
5,M2R,North York,Willowdale West,43.782736,-79.442259
6,M2H,North York,Hillcrest Village,43.803762,-79.363452
7,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.653226,-79.383184
9,M5J,Downtown Toronto,"Toronto Islands, Union Station",43.640816,-79.381752
